In [1]:
!pip install "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git"
!pip install datasets transformers trl peft accelerate bitsandbytes pandas wandb evaluate rouge-score


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-q53qeikx/unsloth_4ce629e381104ed9bb8e642321d4c23d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-q53qeikx/unsloth_4ce629e381104ed9bb8e642321d4c23d
  Resolved https://github.com/unslothai/unsloth.git to commit bcb1bcfd6aa9dc7db8910c733cecf0c676b37eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 11.5 MB/s eta 0:

In [3]:
!pip install unsloth_zoo

In [5]:
!pip uninstall -y torchvision torchtext torchaudio


Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126


In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 48.6 MB/s eta 0:00:00


In [7]:
!pip install "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git"
!pip install unsloth_zoo

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ezy884qx/unsloth_f40d0a9e76c648509a04f1bb52fffe1d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ezy884qx/unsloth_f40d0a9e76c648509a04f1bb52fffe1d
  Resolved https://github.com/unslothai/unsloth.git to commit bcb1bcfd6aa9dc7db8910c733cecf0c676b37eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu124/xformers-0.0.29.post3-cp312-cp312-manylinux_2_28_x86_64.whl (43.4 MB)


In [21]:
import torch
from unsloth import FastLanguageModel
import wandb

# ✅ Login to Weights & Biases
wandb.login()

# Load 4-bit quantized TinyLlama model + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama-chat-bnb-4bit",
    max_seq_length=512,
    load_in_4bit=True,
    use_gradient_checkpointing=True,
)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [22]:
wandb.init(project="tinyllama-it-support-prod", name="initial-run")

data_path = "/content/drive/MyDrive/dataset.csv"
import pandas as pd
df = pd.read_csv(data_path).dropna(subset=["question","answer"])

from datasets import Dataset
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)

# Format dataset for instruction tuning
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['question']}\n\n### Response:\n{example['answer']}"
    }

dataset = dataset.map(format_example)
print(dataset)
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)
print(tokenized_dataset)


train/epoch,▁▁▃▅▆█▁▃▅▆
train/global_step,▁▁▃▅▆█▁▃▅▆
train/grad_norm,█▂▂▂▂▁▁▁▁▁
train/learning_rate,▆█▇▅▄▂▇▅▃▁
train/loss,█▁▁▁▁▁▁▁▁▁
train/epoch,0.35556
train/global_step,200
train/grad_norm,0.21992
train/learning_rate,0.00017
train/loss,0.2492


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 500
    })
})


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text', 'input_ids', 'attention_mask'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['question', 'answer', 'text', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})


In [23]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

In [36]:
from datetime import datetime

# ✅ Customizable unique run ID for W&B
run_id = f"tinyllama-it-support-epochs3-run-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

training_args = TrainingArguments(
    output_dir=f"./{run_id}",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    eval_strategy="steps",
    eval_steps=140,
    save_steps=500,
    save_total_limit=1,
    num_train_epochs=1,             # updated to 3 epochs
    learning_rate=2e-4,
    fp16=True,                      # safer than bf16
    logging_steps=50,
    report_to="wandb",
    run_name=run_id,                # ✅ unique name in W&B
)


In [37]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import evaluate

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-750935174.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [38]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 1 | Total steps = 563
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 12,615,680 of 1,112,664,064 (1.13% trained)


Step,Training Loss,Validation Loss
140,0.255900,0.744887
280,0.245300,0.732229
420,0.249100,0.755370
560,0.244000,0.750267


TrainOutput(global_step=563, training_loss=0.2481725783373492, metrics={'train_runtime': 732.1248, 'train_samples_per_second': 6.146, 'train_steps_per_second': 0.769, 'total_flos': 7237749178368000.0, 'train_loss': 0.2481725783373492, 'epoch': 1.0})

In [40]:
lora_output_dir = "/content/drive/MyDrive/tinyllama-it-support-lora"
model.save_pretrained(lora_output_dir)
tokenizer.save_pretrained(lora_output_dir)

print(f"LoRA adapters saved to: {lora_output_dir}")

LoRA adapters saved to: /content/drive/MyDrive/tinyllama-it-support-lora


In [42]:
# ✅ Correct way in Unsloth
merged_model = model.merge_and_unload()

# Save merged model + tokenizer
merged_output_dir = "/content/drive/MyDrive/tinyllama-it-support-merged"
merged_model.save_pretrained(merged_output_dir, safe_serialization=True)
tokenizer.save_pretrained(merged_output_dir)

print(f" Full merged model saved to: {merged_output_dir}")


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


 Full merged model saved to: /content/drive/MyDrive/tinyllama-it-support-merged


EVALUATION

In [54]:
!pip install peft -q


In [60]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
wandb.init(project="tinyllama-it-support-eval", name="evaluation-run")

merged_model_path = "/content/drive/MyDrive/tinyllama-it-support-merged"

# Use FastLanguageModel to load the merged model
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=merged_model_path, # Load from the saved path
    max_seq_length=512,
    dtype=torch.float16,  # or float32 if no GPU
    load_in_4bit=False, # Load as full precision
    device_map="auto"
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [61]:
data_path = "/content/drive/MyDrive/dataset.csv"   # 👈 same dataset used for training
df = pd.read_csv(data_path).dropna(subset=["question","answer"])
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1, seed=42)
test_dataset = dataset["test"]

print(test_dataset)

Dataset({
    features: ['question', 'answer'],
    num_rows: 500
})


In [69]:
import evaluate

bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")


In [70]:
# === Perplexity function ===
def compute_perplexity(model, tokenizer, references, batch_size=8):
    model.eval()
    losses = []
    for i in range(0, len(references), batch_size):
        batch_refs = references[i:i+batch_size]
        inputs = tokenizer(batch_refs, padding=True, truncation=True, return_tensors="pt").to(model.device)
        labels = inputs.input_ids.clone()
        with torch.no_grad():
            outputs = model(**inputs, labels=labels)
            losses.append(outputs.loss.item())
    mean_loss = sum(losses)/len(losses)
    return torch.exp(torch.tensor(mean_loss)).item()


In [72]:
wandb.init(project="tinyllama-it-support-prod-eval", name="evaluation-run", reinit=True)

# === Batched generation + predictions + live W&B logging ===
predictions, references = [], []
batch_size = 8

for i in range(0, len(test_dataset), batch_size):
    batch = test_dataset.select(range(i, min(i + batch_size, len(test_dataset))))

    # Input formatting
    input_texts = [f"### Instruction:\n{ex['question']}\n\n### Response:\n" for ex in batch]
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # Generate outputs
    outputs = model.generate(**inputs, max_new_tokens=128)

    batch_predictions, batch_references = [], []

    # Decode outputs
    for j, output in enumerate(outputs):
        pred = tokenizer.decode(output, skip_special_tokens=True)
        if "### Response:" in pred:
            pred = pred.split("### Response:")[-1].strip()
        batch_predictions.append(pred)
        batch_references.append(batch[j]["answer"])

    predictions.extend(batch_predictions)
    references.extend(batch_references)

    # Compute batch-wise metrics
    batch_bleu = bleu.compute(predictions=batch_predictions, references=batch_references)["score"]
    batch_rouge = rouge.compute(predictions=batch_predictions, references=batch_references)
    batch_perplexity = compute_perplexity(model, tokenizer, batch_references, batch_size=len(batch_references))

    # Log metrics to W&B
    wandb.log({
        "BLEU": batch_bleu,
        "ROUGE1": batch_rouge["rouge1"],
        "ROUGE2": batch_rouge["rouge2"],
        "ROUGEL": batch_rouge["rougeL"],
        "Perplexity": batch_perplexity
    })
wandb.finish()


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


BLEU,▄▁▄▂▇▂█▃▅▃▃▅▂▂▁▅▄▂▃▂▁▁▁▁▃█▄▅▄▂▂▂▂▂▂▄▁▁▆▄
Perplexity,▂▂▁▂▂▃▂▃▁▂▂▁▃▂▂▃▃▂▃▁▁▁▅█▂▃▄▁▁▂▃█▁▅▂▃▂▂▃▁
ROUGE1,▄▄▇█▄█▃▄▇▂▃█▆▃▄▅▇▅▄▅▃▄▅▃▄▁▄▂▆▅▆▄▅▅▃▃▄▃▃▇
ROUGE2,▄▇█▂▅▂▂▁▁▂▂▁▄▇▆▄▃▁▂▁▅▁▁▃▃▅▂▂▁▃▂▂▄▁▄▁▃▄▃▃
ROUGEL,▃▃█▇██▃▃▆▅▇▅▃▃▂▅▃▅▄▂▃▅▂▃▆▄▁▁▅▃▃▅▅▃▆▃▅▂▄▇
BLEU,0.37774
Perplexity,126.25185
ROUGE1,0.10875
ROUGE2,0.0119
ROUGEL,0.09184


In [74]:
results_df = pd.DataFrame({
    "question": [ex["question"] for ex in test_dataset],
    "reference": references,
    "prediction": predictions
})
results_df.to_csv("/content/drive/MyDrive/tinyllama-it-support-eval.csv", index=False)

print("Evaluation complete. Metrics logged to W&B and CSV saved.")

Evaluation complete. Metrics logged to W&B and CSV saved.


In [76]:
import torch
from unsloth import FastLanguageModel

# Paths
base_model_path = "unsloth/tinyllama-chat-bnb-4bit"   # base TinyLlama
adapter_path = "/content/drive/MyDrive/tinyllama-it-support-lora"  # your LoRA adapter

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_path,
    max_seq_length=512,
    load_in_4bit=True,
    use_gradient_checkpointing=True,
)

# Apply LoRA adapter
model.load_adapter(adapter_path)

# Single input
question = "User cannot access VPN, it keeps failing."

input_text = f"### Instruction:\n{question}\n\n### Response:\n"
inputs = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=128)

# Decode output
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
if "### Response:" in answer:
    answer = answer.split("### Response:")[-1].strip()

print(f"Q: {question}\nA: {answer}")


==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Q: User cannot access VPN, it keeps failing.
A: Check your internet connection, restart the VPN client, and ensure your VPN settings in VPN client
